# main

In [1]:
import json
import requests
import popelines
import os
from main import fix_values, populating_vals, flatten, flatten_dupe_vals
from datetime import datetime

In [2]:
def fix_values(value, key, reset_key):
    if key == reset_key:
        new_list = []
        for x in value:
            value[x][f'{reset_key}_id'] = x
            new_list.append(value[x])
        return new_list
    else:
        return value

In [3]:
# read endpoint, returns a json file of the HTTP request
def read_endpoint(endpoint, headers_set, params_set=None):
    try:
        response = requests.get(endpoint, headers=headers_set, params=params_set)
        response_text = json.loads(response.text)
        response.raise_for_status()

    except requests.exceptions.HTTPError as err:
        print(err)

    return response_text

In [4]:
# # generate all experiments given a project id
# def generate_experiments(project_id, experiment_endpoint, experiment_headers, experiment_params):
#     # get all experiments from one project
#     j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=experiment_headers, params_set=experiment_params)

#     # loop for all experiments in this project
# #     experiment_id_list = []
# #     upload_exp_list = []
# #     for exp in j_exp:
# #         exp['project_id'] = project_id
# #         upload_exp_list.append(flatten(exp, {}, ''))
# #         experiment_id_list.append(exp['id'])
        
#     return upload_exp_list, experiment_id_list

In [5]:
# generate all projects within account
def generate_projects(project_endpoint, project_headers):
    # get all projects
    j_proj = read_endpoint(endpoint=project_endpoint, headers_set=project_headers)

    # store a list of project metadata
    return j_proj

# load all projects, get project_id's

In [6]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
    'Content-Type': 'application/json',
}

In [7]:
project_endpoint = 'https://api.optimizely.com/v2/projects'

In [8]:
all_projects = generate_projects(project_endpoint, headers)

In [ ]:
# pope.write_to_json(file_name='../uploads/projects.json', jayson=all_projects, mode='w')

In [ ]:
# pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
# print("Successfully uploaded all projects.")

In [9]:
project_id_list = []
for project in all_projects:
    project_id_list.append((project['id']))

In [ ]:
# project_id_list


In [ ]:
# project_id_name = []
# for project in all_projects:
#     project_id_name.append((project['id'], project['name']))

In [ ]:
# project_id_name

# load experiments

In [10]:
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'


In [42]:
# experiment_id_list = []
# all_exp_list = []

# for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
params = (
    ('project_id', 10427612860),
    ('per_page', 100),
) 

#     exp_list = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)
#     all_exp_list.extend(exp_list)

In [43]:
exp_list = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)



In [ ]:
# exp_list

In [44]:
experiment_id_list = []

In [45]:
exp_id_list = []
for exp in exp_list:
    exp_id_list.append(exp['id'])

In [46]:
experiment_id_list.extend(exp_id_list)

In [16]:
new = []

In [17]:
for e in exp_list:
    if e['id'] == 12746930280:
        new = e

In [18]:
exp=new

In [47]:
experiment_id_list

[11377370874,
 12107582952,
 12632940119,
 12722820205,
 12743540432,
 12746930280,
 13727000072]

# single layer table

In [ ]:
all_singles = []
nested_key_list = []
for k,v in exp.items():
    if isinstance(v, list):
        nested_key_list.append(k)

single_layer_experiment = {}    
for k,v in exp.items():
    if k not in nested_key_list:
        k = k.replace('-', '_')
        single_layer_experiment[k] = exp[k]
single_layer_experiment['upload_ts'] = str(datetime.now())


all_singles.append(flatten(single_layer_experiment, {}, ''))


In [ ]:
all_singles

# unnested metrics table

In [ ]:
metrics_table = []
for exp in exp_list:
    print(f"Processing experiment {exp['id']}")

    # single layer fields:
    nested_key_list = []
    for k,v in exp.items():
        if isinstance(v, list) or isinstance(v, dict):
            nested_key_list.append(k)

    single_layer_experiment = {}    
    for k,v in exp.items():
        if k not in nested_key_list:
            k = k.replace('-', '_')
            single_layer_experiment[k] = exp[k]
    single_layer_experiment['upload_ts'] = str(datetime.now())

    all_singles.append(flatten(single_layer_experiment, {}, ''))

    # nested part into separate tables:
    # metrics table:

    flattened_metric = []
    for element in exp['metrics']:
        flattened_metric.append(element)

    updated_metric = populating_vals(outer_dict=exp, inner_flattened_list=flattened_metric, destination_key='metrics')
    new_flattened_metric = flatten_dupe_vals(vals=updated_metric, key='metrics')

    metric_list = []
    for metric in new_flattened_metric:
        metric_dict = {}
        metric_dict['metrics_aggregator'] = metric['metrics_aggregator']
        if 'metrics_event_id' in metric.keys():
            metric_dict['metrics_event_id'] = metric['metrics_event_id']
        metric_dict['metrics_scope'] =  metric['metrics_scope']
        metric_dict['metrics_winning_direction'] = metric['metrics_winning_direction']
        metric_dict['experiment_id'] = exp['id']
        metric_dict['upload_ts'] = str(datetime.now())
        metric_list.append(metric_dict)
        
    metrics_table.extend(metric_list)
    

In [ ]:
# nested part into separate tables:
# metrics table:

flattened_metric = []
for element in exp['metrics']:
    flattened_metric.append(element)

updated_metric = populating_vals(outer_dict=exp, inner_flattened_list=flattened_metric, destination_key='metrics')
new_flattened_metric = flatten_dupe_vals(vals=updated_metric, key='metrics')

metric_list = []
for metric in new_flattened_metric:
    metric_dict = {}
    metric_dict['metrics_aggregator'] = metric['metrics_aggregator']
    if 'metrics_event_id' in metric.keys():
        metric_dict['metrics_event_id'] = metric['metrics_event_id']
    metric_dict['metrics_scope'] =  metric['metrics_scope']
    metric_dict['metrics_winning_direction'] = metric['metrics_winning_direction']
    metric_dict['experiment_id'] = exp['id']
    metric_dict['upload_ts'] = str(datetime.now())
    metric_list.append(metric_dict)

In [ ]:
metric_list

# unnested variations table

In [ ]:
variations_table = []
for exp in exp_list:
    print(f"Processing experiment {exp['id']}")

    variations = {}
    # for exp in exp_list:

    print(f"Processing experiment {exp['id']}")

    variations['experiment_id'] = exp['id']
    variations['variations'] = exp['variations']

    flattened_variations = []

    for var in exp['variations']:
        flattened_actions = []
        if len(var['actions']) > 0:
            for action in var['actions']:
                flattened_changes = []
                for element in action['changes']:
                    flattened_changes.append(element)
                # Replace old 'changes' with new 'flattened_changes'
                updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
                new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

                update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
                flat = flatten_dupe_vals(vals=update_actions, key='actions')
                flattened_actions.extend(flat)

        else:
            other_flat = {}
            for k,v in var.items():
                if k != 'actions':
                    other_flat['actions'] = []
                    other_flat[k] = v
            flat = [other_flat]
            flattened_actions.extend(flat)

        update_variations = populating_vals(outer_dict=variations, inner_flattened_list=flattened_actions, destination_key='variations')
        flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

    variations_table.extend(flattened_variations)
    
    

In [ ]:
variations = {}
# for exp in exp_list:

print(f"Processing experiment {exp['id']}")

variations['experiment_id'] = exp['id']
variations['variations'] = exp['variations']



In [ ]:
exp['variations'][0]


In [ ]:
flattened_variations = []

for var in exp['variations']:
    flattened_actions = []  
    
    if len(var['actions']) > 0:
        for action in var['actions']:

            flattened_changes = []

            if action['changes'] != []:
                for element in action['changes']:
                    flattened_changes.append(element)
                # Replace old 'changes' with new 'flattened_changes'
                updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
                new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

                update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
                flat = flatten_dupe_vals(vals=update_actions, key='actions')
                flattened_actions.extend(flat)

            else:
                new_actions = flatten_dupe_vals(vals=var, key='actions')
                flattened_actions.extend(new_actions)
            
            print('completed round 1')
            
    else:
        other_flat = {}
        for k,v in var.items():
            if k != 'actions':
                other_flat['actions'] = []
                other_flat[k] = v
        flat = [other_flat]
        flattened_actions.extend(flat)

        print('completed round 2')

    update_variations = populating_vals(outer_dict=variations, inner_flattened_list=flattened_actions, destination_key='variations')
    flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

    
    

In [ ]:
flattened_variations

### Testing flatten_dupe_vals

In [ ]:
# new_flattened_test = flatten_dupe_vals(vals=test, key='actions')
vals = test
key = 'actions'

In [ ]:
duped_results = []
for element in vals[key]:
    ts_dict = populating_vals(outer_dict=vals, inner_flattened_list=element, destination_key=key)
    duped_results.append(ts_dict)



In [ ]:
duped_results

In [ ]:
flattened_results = []
for element in duped_results:
    flattened_results.append(flatten(element, {}, ''))

In [ ]:
flatten(duped_results[0], {}, '')

### It is the flatten function that f*cked things up

In [ ]:
duped_results[0]

In [ ]:
# corrected flatten
def flatten(jayson, acc, prefix):
    if isinstance(jayson, dict):
        for k,v in jayson.items():
            if prefix:
                prefix_k = prefix + "_" + k
            else: 
                prefix_k = k
            prefix_k = prefix_k.replace('-', '_')
            
            if isinstance(v, dict):
                flatten(v, acc, prefix_k)
            elif isinstance(v, list):
                if v != []:
                    for j in v:
                        flatten(j, acc, prefix_k)
                else:
                    acc[prefix_k] = v
            else:
                acc[prefix_k] = v
        return acc 
    else:
        return acc

In [ ]:
flatten(duped_results[0], {}, '')

### Thus fixing the missing changes issue!

In [ ]:
flattened_variations = []

for var in expanded_exp['variations']:
    flattened_actions = []
    for action in var['actions']:
        print(action)
#         flattened_changes = []
#         for element in action['changes']:
#             flattened_changes.append(element)

        # Replace old 'timeseries' with new 'flattened_timeseries'
#         updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
#         new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

#     update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
#     flattened_actions.extend(flatten_dupe_vals(vals=update_actions, key='actions'))  

# update_variations = populating_vals(outer_dict=expanded_exp, inner_flattened_list=flattened_actions, destination_key='variations')
# flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

# expanded_variations = flattened_variations

In [ ]:
flattened_changes

# results timeseries

In [ ]:
# experiment_id_list

In [ ]:
# example of no results
# result_endpoint = 'https://api.optimizely.com/v2/experiments/11479333433/timeseries'
# example of normal time series
result_endpoint = 'https://api.optimizely.com/v2/experiments/12813120687/timeseries'
# example of 
# result_endpoint = f'https://api.optimizely.com/v2/experiments/{experiment_id}/timeseries'



In [ ]:
# for experiment_id in experiment_id_list:
# result_endpoint = f'https://api.optimizely.com/v2/experiments/{experiment_id}/timeseries'
response_ts = requests.get(result_endpoint, headers=headers)
print(f"got experiment 12813120687")
    
    

In [ ]:
# if '' then the experiment has not started yet
if response_ts.text == '' or 'bad' in response_ts.text:
    j_ts = {'experiment_id': experiment_id}
    new_j_ts = j_ts
    new_j_ts['upload_ts'] = str(datetime.now())
    
else:
    j_ts = json.loads(response_ts.text)

    new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')

    flattened_j_ts = []

    flattened_metrics = []
    for metric in new_j_ts['metrics']:
        if 'results' in metric.keys():
            
            for ts in metric['results']:
                flattened_timeseries = []
                for element in ts['timeseries']:
                    element['upload_ts'] = str(datetime.now())
                    flattened_timeseries.append(flatten(element, {}, ''))

                # Replace old 'timeseries' with new 'flattened_timeseries'
                updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
                flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')

            # Replace old 'metrics' with new 'flattened_results'
            update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
            flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))

        else:
            flattened_metrics = [flatten(new_j_ts, {}, '')]

    update_new_j_ts = populating_vals(outer_dict=new_j_ts, inner_flattened_list=flattened_metrics, destination_key='metrics')
    flattened_j_ts.extend(flatten_dupe_vals(vals=update_new_j_ts, key='metrics'))
    
    

In [ ]:
j_ts = json.loads(response_ts.text)

new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')





In [ ]:
new_j_ts['metrics']

In [ ]:
flattened_j_ts = []

flattened_metrics = []
for metric in new_j_ts['metrics']:
    if 'results' in metric.keys():
        flattened_results = []
        for ts in metric['results']:
            flattened_timeseries = []
            for element in ts['timeseries']:
                element['upload_ts'] = str(datetime.now())
                flattened_timeseries.append(flatten(element, {}, ''))

            # Replace old 'timeseries' with new 'flattened_timeseries'
            updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
            flattened_results.extend(flatten_dupe_vals(vals=updated_results, key='timeseries'))

        # Replace old 'metrics' with new 'flattened_results'
        update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
        flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))

    else:
        flattened_metrics = [flatten(new_j_ts, {}, '')]

update_new_j_ts = populating_vals(outer_dict=new_j_ts, inner_flattened_list=flattened_metrics, destination_key='metrics')
flattened_j_ts.extend(flatten_dupe_vals(vals=update_new_j_ts, key='metrics'))


In [ ]:

# for ts in new_j_ts['metrics'][0]['results']:
#     flattened_timeseries = []
#     for element in ts['timeseries']:
#         element['upload_ts'] = str(datetime.now())
#         flattened_timeseries.append(flatten(element, {}, ''))
#         break

#     # Replace old 'timeseries' with new 'flattened_timeseries'
#     updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
#     flattened_results.extend(flatten_dupe_vals(vals=updated_results, key='timeseries'))
#     break

In [ ]:
len(flattened_j_ts)

In [ ]:
output = []
for e in flattened_j_ts:
    if e['metrics_name'] == 'Modal open button' and e['metrics_results_timeseries_time'] == '2019-01-08T23:29:23.000000Z':
        output.append(e)
        
        

In [ ]:
output


In [ ]:
length = 0
for e in new_j_ts['metrics'][0]['results']:
    print(f"{e['name']} is of {len(e['timeseries'])} long")

In [ ]:
length

In [ ]:
for e in flattened_j_ts:
    print(e['metrics_results_name'])

# testing

In [ ]:
from generate_original import read_endpoint, generate_projects, generate_experiments

In [ ]:
if not os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS'):
    os.environ['GOOGLE_ACCOUNT_CREDENTIALS'] = '/home/engineering/keyfile.json'
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

############################################### Instantiating Popelines #######################################
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

# endpoints
project_endpoint = 'https://api.optimizely.com/v2/projects'
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'


############################################### generate and upload all projects ##############################
all_projects = generate_projects(project_endpoint, headers)
for project in all_projects:
    project['upload_ts'] = str(datetime.now())

# upload projects 
pope.write_to_json(file_name='../uploads/projects.json', jayson=all_projects, mode='w')
pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)


############################################### generate and upload all experiments ##############################
# get a list of project_id from all_projects
project_id_list = []
for project in all_projects:
    project_id_list.append(project['id'])

# to accumulate all experiment_id, for 
experiment_id_list = []
origin_single_table = []
origin_metrics_table = []
origin_variations_table = []

# loop over all project_id_list to get experiments within each project
for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
    params = (
        ('project_id', project_id),
        ('per_page', 100),
    ) 

    exp_list = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)
    exp_id_list = []
    for exp in exp_list:
        exp_id_list.append(exp['id'])
    experiment_id_list.extend(exp_id_list)

    all_singles, metrics_table, variations_table = generate_experiments(exp_list)
    origin_single_table.extend(all_singles)
    origin_metrics_table.extend(metrics_table)
    origin_variations_table.extend(variations_table)

pope.write_to_json(file_name='../uploads/origin_experiments_single_fields.json', jayson=origin_single_table, mode='w')  
pope.write_to_json(file_name='../uploads/origin_experiments_metrics_table.json', jayson=origin_metrics_table, mode='w')
pope.write_to_json(file_name='../uploads/origin_experiments_variations_table.json', jayson=origin_variations_table, mode='w')





In [ ]:
for e in origin_single_table:
    if e['id'] == 12900000180:
        print(e)

In [ ]:
for m in origin_metrics_table:
    if m['experiment_id'] == 12900000180:
        print(m)

In [ ]:
l = []
for v in origin_variations_table:
    if v['experiment_id'] == 12900000180:
        l.append(v['variations_name'])

In [ ]:
l

In [ ]:
exp_list

In [ ]:
for e in exp_list:
    print(e['id'])

In [ ]:
exp

In [ ]:
variations = {}
variations['experiment_id'] = exp['id']
variations['variations'] = exp['variations']

flattened_variations = []

for var in exp['variations']:
    flattened_actions = []
    if len(var['actions']) > 0:
        for action in var['actions']:
            flattened_changes = []
            for element in action['changes']:
                flattened_changes.append(element)
            # Replace old 'changes' with new 'flattened_changes'
            updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
            new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

        update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
        flat = flatten_dupe_vals(vals=update_actions, key='actions')
        
        flattened_actions.extend(flat)

    else:
        other_flat = {}
        for k,v in var.items():
            if k != 'actions':
                other_flat['actions'] = []
                other_flat[k] = v
        flat = [other_flat]
    
        flattened_actions.extend(flat)

    update_variations = populating_vals(outer_dict=variations, inner_flattened_list=flattened_actions, destination_key='variations')
    flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))
    

In [ ]:
len(flattened_variations)

In [ ]:
for v in flattened_variations:
    print(v['variations_variation_id'])

In [ ]:
for v in flattened_variations:
    print(v['variations_name'])

In [ ]:
for v in flattened_variations:
    if 'variations_actions_changes_id' in v.keys():
        print(v['variations_actions_changes_id'])

# Testing Result timeseries

In [20]:
# exp_id = 12746930280
# no result: exp_id = 12632940119

In [202]:
experiment_id_list = [12746930280, 12632940119]

In [203]:
# loop over all experiment_id in experiment_id_list from above
for experiment_id in experiment_id_list:
    result_endpoint = f'https://api.optimizely.com/v2/experiments/{experiment_id}/timeseries'
    response_ts = requests.get(result_endpoint, headers=headers)
    print(f"got experiment {experiment_id}")
    # if '' then the experiment has not started yet
    if response_ts.text == '' or 'bad' in response_ts.text:
        j_ts = {'experiment_id': experiment_id}
        new_j_ts = j_ts
        new_j_ts['upload_ts'] = str(datetime.utcnow())
        pope.write_to_json(file_name='../uploads/testing_no_results.json', jayson=[new_j_ts], mode='w')
        pope.write_to_bq(table_name='testing_results', file_name='../uploads/testing_no_results.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
            
        
    else:
        j_ts = json.loads(response_ts.text)

        new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')

        flattened_j_ts = []
        flattened_metrics = []
        
        for metric in new_j_ts['metrics']:
            
            if 'results' in metric.keys():
                for ts in metric['results']:
                    flattened_results = []

                    for element in ts['timeseries']:
                        flattened_timeseries = []
                        element['upload_ts'] = str(datetime.utcnow())
                        flattened_timeseries.append(flatten(element, {}, ''))

                        # Replace old 'timeseries' with new 'flattened_timeseries'
                        updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
                        flattened_results.extend(flatten_dupe_vals(vals=updated_results, key='timeseries'))

                # Replace old 'metrics' with new 'flattened_results'
                update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
                flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))
                
            else:
                flattened_metrics = [flatten(new_j_ts, {}, '')]
                
        update_new_j_ts = populating_vals(outer_dict=new_j_ts, inner_flattened_list=flattened_metrics, destination_key='metrics')
        flattened_j_ts.extend(flatten_dupe_vals(vals=update_new_j_ts, key='metrics'))

        pope.write_to_json(file_name='../uploads/testing_results.json', jayson=flattened_j_ts, mode='w')
        pope.write_to_bq(table_name='testing_results', file_name='../uploads/testing_results.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
        
            


INFO:root:Writing testing_results to BQ from file ../uploads/testing_results.json


got experiment 12746930280
2019-02-25 11:49:01,412 - root - Writing testing_results to BQ from file ../uploads/testing_results.json


INFO:root:Writing testing_results to BQ from file ../uploads/testing_no_results.json


got experiment 12632940119
2019-02-25 11:49:06,708 - root - Writing testing_results to BQ from file ../uploads/testing_no_results.json


In [188]:
new_j_ts

{'experiment_id': 12632940119, 'upload_ts': '2019-02-25 19:42:35.276053'}

In [201]:
for e in flattened_j_ts:
    print(e['metrics_name'], e['metrics_results_timeseries_time'])

CTA - View Guide Now 2019-01-07T23:54:30.000000Z
CTA - View Guide Now 2019-01-08T11:37:36.300000Z
CTA - View Guide Now 2019-01-08T23:20:42.600000Z
CTA - View Guide Now 2019-01-09T11:03:48.900000Z
CTA - View Guide Now 2019-01-09T22:46:55.200000Z
CTA - View Guide Now 2019-01-10T10:30:01.500000Z
CTA - View Guide Now 2019-01-10T22:13:07.800000Z
CTA - View Guide Now 2019-01-11T09:56:14.100000Z
CTA - View Guide Now 2019-01-11T21:39:20.400000Z
CTA - View Guide Now 2019-01-12T09:22:26.700000Z
CTA - View Guide Now 2019-01-12T21:05:33.000000Z
CTA - View Guide Now 2019-01-13T08:48:39.300000Z
CTA - View Guide Now 2019-01-13T20:31:45.600000Z
CTA - View Guide Now 2019-01-14T08:14:51.900000Z
CTA - View Guide Now 2019-01-14T19:57:58.200000Z
CTA - View Guide Now 2019-01-15T07:41:04.500000Z
CTA - View Guide Now 2019-01-15T19:24:10.800000Z
CTA - View Guide Now 2019-01-16T07:07:17.100000Z
CTA - View Guide Now 2019-01-16T18:50:23.400000Z
CTA - View Guide Now 2019-01-17T06:33:29.700000Z
CTA - View Guide Now

In [190]:
result_endpoint = 'https://api.optimizely.com/v2/experiments/12746930280/timeseries'
response_ts = requests.get(result_endpoint, headers=headers)
print("got experiment 12746930280")

   

got experiment 12746930280


In [191]:
j_ts = json.loads(response_ts.text)

new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')

    

In [192]:
new_j_ts

{'confidence_threshold': 0.95,
 'end_time': '2019-02-25T19:40:00.000000Z',
 'experiment_id': 12746930280,
 'metrics': [{'aggregator': 'unique',
   'event_id': 12826470463,
   'name': 'CTA - View Guide Now',
   'results': [{'is_baseline': False,
     'level': 'variation',
     'name': 'Variation #1',
     'results_id': '12762280184',
     'timeseries': [{'lift': {'is_significant': False,
        'lift_status': 'equal',
        'significance': 0.0,
        'visitors_remaining': 9223372036854775807},
       'samples': 0,
       'time': '2019-01-07T23:54:30.000000Z',
       'value': 0,
       'variance': 0.0},
      {'lift': {'is_significant': False,
        'lift_status': 'better',
        'significance': 0.0,
        'value': 0.20634920634920628,
        'visitors_remaining': 9223372036854775807},
       'rate': 0.12698412698412698,
       'samples': 63,
       'time': '2019-01-08T11:37:33.300000Z',
       'value': 8,
       'variance': 0.11085915847820609},
      {'lift': {'is_significa

In [197]:
flattened_j_ts = []

flattened_metrics = []
for metric in new_j_ts['metrics']:
    
    if 'results' in metric.keys():
        for ts in metric['results']:
            flattened_results = []

            for element in ts['timeseries']:
                flattened_timeseries = []
                element['upload_ts'] = str(datetime.utcnow())
                flattened_timeseries.append(flatten(element, {}, ''))

                # Replace old 'timeseries' with new 'flattened_timeseries'
                updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
                flattened_results.extend(flatten_dupe_vals(vals=updated_results, key='timeseries'))

        # Replace old 'metrics' with new 'flattened_results'
        update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
        flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))

    else:
        flattened_metrics = [flatten(new_j_ts, {}, '')]

update_new_j_ts = populating_vals(outer_dict=new_j_ts, inner_flattened_list=flattened_metrics, destination_key='metrics')
flattened_j_ts.extend(flatten_dupe_vals(vals=update_new_j_ts, key='metrics'))

# pope.write_to_json(file_name='../uploads/testing_results.json', jayson=flattened_j_ts, mode='w')
    
    

In [198]:
for e in flattened_j_ts:
    print(e['metrics_name'], e['metrics_results_timeseries_time'])

CTA - View Guide Now 2019-01-07T23:54:30.000000Z
CTA - View Guide Now 2019-01-08T11:37:33.300000Z
CTA - View Guide Now 2019-01-08T23:20:36.600000Z
CTA - View Guide Now 2019-01-09T11:03:39.900000Z
CTA - View Guide Now 2019-01-09T22:46:43.200000Z
CTA - View Guide Now 2019-01-10T10:29:46.500000Z
CTA - View Guide Now 2019-01-10T22:12:49.800000Z
CTA - View Guide Now 2019-01-11T09:55:53.100000Z
CTA - View Guide Now 2019-01-11T21:38:56.400000Z
CTA - View Guide Now 2019-01-12T09:21:59.700000Z
CTA - View Guide Now 2019-01-12T21:05:03.000000Z
CTA - View Guide Now 2019-01-13T08:48:06.300000Z
CTA - View Guide Now 2019-01-13T20:31:09.600000Z
CTA - View Guide Now 2019-01-14T08:14:12.900000Z
CTA - View Guide Now 2019-01-14T19:57:16.200000Z
CTA - View Guide Now 2019-01-15T07:40:19.500000Z
CTA - View Guide Now 2019-01-15T19:23:22.800000Z
CTA - View Guide Now 2019-01-16T07:06:26.100000Z
CTA - View Guide Now 2019-01-16T18:49:29.400000Z
CTA - View Guide Now 2019-01-17T06:32:32.700000Z
CTA - View Guide Now